# <img src="./logo_transparent.png" width="70"> Step3 - Planning des matchs <img src="./logoffhb.png" width="100"> 

In [1]:
import pandas as pd
import sqlite3
#Creation des invitation ics
from icalendar import Calendar, Event
import pytz
from datetime import datetime
import os
# from pathlib import Path
import time
import ipywidgets as widgets
from IPython.display import display

In [2]:
#start = time.time() # To measure time
pd.set_option('display.max_rows', None) # Show all raws from pandas df

In [3]:
conn = sqlite3.connect('Championnat_U18_GrandEst.sqlite')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM Matchs", conn)

In [4]:
cal = Calendar()
#cal.add('attendee', 'MAILTO:magali.capelli@garrettmotion.com')
#cal.add('attendee', 'MAILTO:xyz@example.com')

In [5]:
def add_event(J,D,M,A):
    # print(J,D,M,A)
    event = Event()
    event.add('summary', M.replace(".pdf","")+D)
    when = D.split()
    #print(when)
    month = {'janvier':1,'février':2,'mars':3,'avril':4,'mai':5,'juin':6,'juillet':7,'aout':8,'septembre':9,'octobre':10,'novembre':11,'dÃ©cembre':12 }
    if month[when[2]] > 8:
        year = datetime.today().year
    else :
        year = datetime.today().year +1
    heure = when[3].split('H')
    event.add('attendee',"magali.capelli@garrettmotion.com")
    event.add('dtstart', datetime(year, month[when[2]], int(when[1]), int((when[3].split('H'))[0])-1, int((when[3].split('H'))[1]), 0, tzinfo=pytz.timezone("Europe/Paris")))
    event.add('dtend', datetime(year, month[when[2]], int(when[1]), int((when[3].split('H'))[0])-1+2, int((when[3].split('H'))[1]), 0, tzinfo=pytz.timezone("Europe/Paris")))
    #event.add('dtstamp', datetime(year, 4, 4, 0, 10, 0, tzinfo=pytz.timezone("Europe/Paris")))
    event.add('location', A)
    # Adding events to calendar
    #print(event)
    cal.add_component(event)

In [6]:
# DEFINITION DE LA LISTE DES EQUIPE
Liste_equipe =[]
for e in df['equipe_dom'].unique():
    if e not in Liste_equipe: Liste_equipe.append(e)
for e in df['equipe_ext'].unique():
    if e not in Liste_equipe: Liste_equipe.append(e)
Liste_equipe.append('Toutes les équipes')

In [7]:
Equipe = 'EPINAL U18M'
message_start = f"""
          <head>
          <meta http-equiv="Content-Type" content="text/html; charset=latin-1">
          <title>Planning des Matchs</title>"""
message_style = """
          <style type="text/css" media="screen">
            #customers {
              font-family: "Trebuchet MS", Arial, Helvetica, sans-serif;
              font-size: 14px;
              border-collapse: collapse;
              width: 100%;
            }
            #customers td, #customers th {
              border: 1px solid #ddd;
              padding: 8px;
            }
            #customers tr:nth-child(even){background-color: #f2f2f2;}
            #customers tr:hover {background-color: #ddd;}
            #customers th {
              padding-top: 12px;
              padding-bottom: 12px;
              text-align: left;
              background-color: #003d77;
              color: white;
            }
          </style>
        </head>
        <body>
        """
titre = f'''
        <h3 align="center" style="font-size:16px font-family:Arial"><b>Journées de Championnat de l'équipe {Equipe}</b></h3>
        '''

In [8]:
df=df.replace('—','')
df=df.replace('— —','')
df=df.replace('— — —','')
df[['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']]
message_body = df[['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']].to_html(index=False, table_id="customers") #set table_id to your css style name
message_end = """</body>"""
message = (message_start + message_style + message_body + message_end)
# OUTPUT AN HTML FILE
with open(f'./PLANNING/Planning_des_Matchs.html', 'w') as f:
        f.write(message)
    # f.write(html_string.format(table=df[['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']].to_html(classes='mystyle',index=False,na_rep='-')))

In [9]:
print(f'Le nombre total de matchs plannifié est de: {df[df["lieu"]!=""].shape[0]}')
print(f'''Le nombre de matchs plannifié pour EPINAL est de: 
    à domicile: {df[(df["lieu"]!="") & (df["equipe_dom"]=="EPINAL U18M")].shape[0]} 
    à l'extérieur: {df[(df["lieu"]!="") & (df["equipe_ext"]=="EPINAL U18M")].shape[0]}''')

Le nombre total de matchs plannifié est de: 46
Le nombre de matchs plannifié pour EPINAL est de: 
    à domicile: 4 
    à l'extérieur: 4


In [10]:
def show_planning(Equipe='EPINAL U18M'):
    if Equipe != 'Toutes les équipes':
        df2 = df[(df['equipe_dom']==Equipe) | (df['equipe_ext']==Equipe)][['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']]
        df2.sort_values(by=['Journee'],ascending=True, inplace=True)
        display(df2.style.hide_index())
        for i in df2.index:
            try:
                # print(df.loc[i]['Journee'],df.loc[i]['Date'],df.loc[i]['fichier'],df.loc[i]['lieu'])
                add_event(df.loc[i]['Journee'],df.loc[i]['Date'],df.loc[i]['fichier'],df.loc[i]['lieu'])
            except:
                pass
        f= open('./PLANNING/'+Equipe+'_Planning_des_Matchs.ics','wb')
        f.write(cal.to_ical())
        f.close()
        pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>       
        message_body = df2.to_html(index=False, table_id="customers") #set table_id to your css style name
        message_end = """</body>"""
        message = (message_start + message_style +titre + message_body + message_end)
        # print(message)
        # OUTPUT AN HTML FILE
        with open(f'./PLANNING/{Equipe}_Planning_des_Matchs.html', 'w') as f:
            f.write(message)
    else:
        display(df[['Journee','Periode','Date','equipe_dom','score_dom','score_ext','equipe_ext','lieu','arbitres']].sort_values(by=['Journee'],ascending=True, inplace=False).style.hide_index())
        

In [11]:
widgets.interact(show_planning, Equipe=Liste_equipe)

interactive(children=(Dropdown(description='Equipe', index=1, options=('STRASBOURG&SCHILTIGHEIM U18M2', 'EPINA…

<function __main__.show_planning(Equipe='EPINAL U18M')>

In [12]:
#print('Task completed in', time.time() - start,'seconds')